In [1]:
import os
import sys
import numpy as np
from librosa import load
from IPython.display import Audio
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from loudness_encoder import loudness

c:\Users\nicho\Desktop\DWST\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nicho\Desktop\DWST\venv\lib\site-packages\nnAudio\Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


In [2]:
signal, sr = load('../test_audio/mallet_acoustic_074-072-050.wav', sr=None)

In [3]:
loud = loudness(signal, sr, 2048, 512)

STFT kernels created, time used = 0.1426 seconds
(126,)


c:\Users\nicho\Desktop\DWST\venv\lib\site-packages\librosa\core\convert.py:1332: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


In [8]:
print(loud)

[-0.4887616  -0.58308418 -0.39396979 -0.45514023 -0.58665327 -0.65151496
 -0.41690711 -1.22961666 -0.41956824 -1.02977161 -0.44759753 -0.66307724
 -0.56913343 -0.50522737 -0.82777359 -0.4460946  -1.2955806  -0.47380767
 -0.76152581 -0.58488273 -0.58133672 -0.80701388 -0.51274574 -1.42136538
 -0.51714672 -0.89398709 -0.59661722 -0.66038709 -0.77580988 -0.56904612
 -1.19644779 -0.55554615 -1.02969291 -0.61489233 -0.74571382 -0.76319375
 -0.62703795 -1.08836152 -0.59591267 -1.18771657 -0.63759795 -0.83017624
 -0.75991796 -0.68760944 -1.01468065 -0.64020672 -1.39181777 -0.6624413
 -0.92419579 -0.75968384 -0.7520357  -0.96827097 -0.68503899 -1.49697244
 -0.69196162 -1.0312727  -0.76913331 -0.82300862 -0.93840685 -0.73682199
 -1.31834723 -0.72521266 -1.1477018  -0.78514942 -0.89525806 -0.92693363
 -0.79006222 -1.22388237 -0.763981   -1.28048205 -0.80586318 -0.97708306
 -0.9217946  -0.84750095 -1.16558648 -0.80595086 -1.435002   -0.82915669
 -1.06156589 -0.92526024 -0.90621594 -1.11907873 -0.

In [4]:
import torch
import numpy as np
import librosa as li
def extract_loudness(audio, sampling_rate, block_size=None, n_fft=2048, frame_rate=None):
    assert (block_size is None) != (frame_rate is None), "Specify exactly one of block_size or frame_rate"

    if frame_rate is not None:
        block_size = sampling_rate // frame_rate
    else:
        frame_rate = int(sampling_rate / block_size)

    if sampling_rate % frame_rate != 0:
        raise ValueError(
            'frame_rate: {} must evenly divide sample_rate: {}.'
            'For default frame_rate: 250Hz, suggested sample_rate: 16kHz or 48kHz'
            .format(frame_rate, sampling_rate))

    if isinstance(audio, np.ndarray):
        audio = torch.tensor(audio)

    # Temporarily a batch dimension for single examples.
    is_1d = (len(audio.shape) == 1)
    audio = audio[None, :] if is_1d else audio

    # Take STFT.
    overlap = 1 - block_size / n_fft
    amplitude = torch.stft(audio, n_fft=n_fft, hop_length=block_size, center=True, pad_mode='reflect', return_complex=True).abs()
    amplitude = amplitude[:, :, :-1]
    
    # Compute power.
    power_db = amplitude_to_db(amplitude)

    # Perceptual weighting.
    frequencies = li.fft_frequencies(sr=sampling_rate, n_fft=n_fft)
    a_weighting = li.A_weighting(frequencies)[None,:,None]
    loudness = power_db + a_weighting

    loudness = torch.mean(torch.pow(10, loudness / 10.0), axis=1)
    loudness = 10.0 * torch.log10(torch.clamp(loudness, min=1e-20))

    # Remove temporary batch dimension.
    loudness = loudness[0] if is_1d else loudness
    loudness = loudness.numpy()

    return loudness

In [5]:
extract_loudness(signal, sr, 512,2048)

ValueError: frame_rate: 31 must evenly divide sample_rate: 16000.For default frame_rate: 250Hz, suggested sample_rate: 16kHz or 48kHz